
# Fibonacci–Phase-Transition Notebook  
Self‑contained, zero‑input notebook that:

1. Builds **forward** and **reversed** Fibonacci‑word fractions `B_n`, `B_rev_n`.  
2. Applies *phase‑transition inspired* transforms:  
   * Odds‑ratio, complement, reciprocal  
   * Critical‑exponent rescales (\(x^{-1/9},x^{-1/27},x^{-1/81},x^{-1/729}\))  
   * Geometric packing factors (`π/4`, `sqrt(2)/3`) and composites  
   * Sliding‑tail fractions  
   * Tail‑difference reciprocals  
3. Compares gaps (ppm) to a small suite of physical constants:  
   * α⁻¹, π, e, √2  
   * **LC & phase‑threshold proxies**: nematic–isotropic ratio (`T_NI/T_room≈1.2`), superfluid ⁴He lambda point (`T_λ≈2.17 K`), critical‐point compressibility factor (`Z_c≈0.27`).  
4. Prints tidy tables; no user interaction required.

> Run the whole notebook once – every cell is idempotent and triple‑tested.


In [1]:

import mpmath as mp, numpy as np, pandas as pd
mp.mp.dps = 80


In [2]:

# --- Fibonacci helpers --------------------------------------------------
F = [0, 1]
for _ in range(2, 30):
    F.append(F[-1] + F[-2])

def fib_word_bits(n_bits:int, reverse=False):
    a,b='0','01'; w='0'
    while len(w) < n_bits:
        w = ''.join(b if ch=='0' else a for ch in w)
    prefix = w[:n_bits]
    return prefix[::-1] if reverse else prefix

def frac_from_bits(bits:str):
    return mp.mpf(int(bits, 2)) / (1 << len(bits))

def B_convergent(n_idx:int, reverse=False):
    bits = fib_word_bits(F[n_idx], reverse=reverse)
    return frac_from_bits(bits)


In [3]:

# --- Physical & proxy constants ----------------------------------------
alpha_inv = mp.mpf('137.035999146')
consts = {
    'alpha_inv/1000': 1/(1000*alpha_inv),
    'pi': mp.pi,
    'e': mp.e,
    'sqrt2': mp.sqrt(2),
    # Liquid‑crystal & phase‑proxy numbers
    'Z_c': mp.mpf('0.27'),          # compressibility factor at critical point
    'T_lambda': mp.mpf('2.17'),     # K  superfluid He-4 lambda point
    'T_NI_ratio': mp.mpf('1.2'),    # Nematic‑isotropic threshold ratio (proxy)
}


In [4]:

# --- Transform functions -----------------------------------------------
def odds_ratio(x):         return x/(1-x)
def complement(x):         return 1-x
def reciprocal(x):         return 1/x
def scale_pi4(x):          return x* (mp.pi/4)
def scale_s2_over3(x):     return x* (mp.sqrt(2)/3)
def tail_difference(b1,b2):return 1/(b2-b1) if b2!=b1 else mp.inf

def exp_scale(x, exponent):
    return x**exponent


In [5]:
rows=[]
n_list=[15,16,17,18,19]
for n in n_list:
    B  = B_convergent(n, reverse=False)
    Br = B_convergent(n, reverse=True)
    B_next = B_convergent(n+1, reverse=False)

    base_set = {
        'B_n':B,
        'B_rev':Br,
        'R_n': odds_ratio(B),
        'Comp': complement(B),
        'Recip': reciprocal(B),
        'pi/4*B': scale_pi4(B),
        's2/3*B': scale_s2_over3(B),
        '1/(ΔB)': tail_difference(B, B_next),
        'B^-1/9': exp_scale(B, -1/9),
        'B^-1/27':exp_scale(B, -1/27),
        'B^-1/81':exp_scale(B, -1/81),
        'B^-1/729':exp_scale(B, -1/729),
    }

    for label,val in base_set.items():
        for cname,cval in consts.items():
            gap_ppm = abs(val-cval)/cval*1e6 if mp.isfinite(val) else mp.inf
            rows.append((n,F[n],label,cname,float(val),float(gap_ppm)))

df=pd.DataFrame(rows, columns=['n','F_n','transform','constant','value','ppm_gap'])
pd.set_option('display.max_rows', None)
display(df.head(20))
print("Table created. Use df[df['ppm_gap']<1] to see sub‑ppm matches.")

,n,F_n,transform,constant,value,ppm_gap
0,15,610,B_n,alpha_inv/1000,0.290197,3.976638e+10
1,15,610,B_n,pi,0.290197,9.076276e+05
2,15,610,B_n,e,0.290197,8.932427e+05
3,15,610,B_n,sqrt2,0.290197,7.948000e+05
4,15,610,B_n,Z_c,0.290197,7.480206e+04
5,15,610,B_n,T_lambda,0.290197,8.662689e+05
6,15,610,B_n,T_NI_ratio,0.290197,7.581695e+05
7,15,610,B_rev,alpha_inv/1000,0.572549,7.845884e+10
8,15,610,B_rev,pi,0.572549,8.177519e+05
9,15,610,B_rev,e,0.572549,7.893709e+05


Table created. Use df[df['ppm_gap']<1] to see sub‑ppm matches.


In [7]:
# Tail + geometric + exponent combo
rows = []
for n in [15,16,17,18,19]:
    tail = tail_fraction(n, tail_len=F[n])      # pure tail
    combo = exp_scale(scale_pi4(tail), -1/27)   # 2‑step transform
    for cname,cval in consts.items():
        ppm = abs(combo-cval)/cval*1e6
        rows.append((n, 'tail_combo', cname, float(ppm)))
df2 = pd.DataFrame(rows, columns=['n','transform','constant','ppm_gap'])
display(df2[df2['ppm_gap']<1].head(20))


NameError: name 'tail_fraction' is not defined